In [1]:
!pip install catboost
!pip install shap
!pip install plotly
!pip install gdown
!pip install PrettyTable
!pip install seaborn
!pip install gdown


In [2]:
import numpy as np
import pandas as pd
import requests
import io
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import confusion_matrix
from prettytable import PrettyTable
from sklearn.metrics import accuracy_score

import random
import requests
import io

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gdown
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# **Fairness Functions**

In [3]:
from sklearn.metrics import confusion_matrix
from prettytable import PrettyTable
from itertools import combinations

def fair_Demographic_Parity(X,y_var,sensitive_var,prediction):
#   print("----------------------------------------------------")
#   print("\t\t\t",sensitive_var)
#   print('----------------------------------------------------')
  groups = X[sensitive_var].unique()
  positive_proportions = []

  # Create a PrettyTable object
  table = PrettyTable()
  table.field_names = ["Name", "proportion_positive"]

  for group in groups:
    group_data = X[X[sensitive_var] == group]

    y_pred_group = group_data[prediction]
    # Calculate Proportion Positive for the current group
    proportion_positive = y_pred_group.mean()

    # Append Proportion Positive for the current group
    positive_proportions.append(proportion_positive)
    table.add_row([group, proportion_positive])
    # Find the maximum absolute difference in Proportion Positive across groups
  max_diff = max(abs(pp1 - pp2) for pp1, pp2 in combinations(positive_proportions, 2))

#   print("Demographic Parity Score :",max_diff)
#   print(table)
  return max_diff

def fair_Eq_Odds(X,y_var,sensitive_var,prediction):
#   print("----------------------------------------------------")
#   print("\t\t\t",sensitive_var)
#   print('----------------------------------------------------')
  groups = X[sensitive_var].unique()
  equal_odds_scores = []

  # Create a PrettyTable object
  table = PrettyTable()
  table.field_names = ["Name", "TPR", "FPR"]

  for group in groups:
    group_data = X[X[sensitive_var] == group]
    y_true_group = group_data[y_var]
    y_pred_group = group_data[prediction]

    cm = confusion_matrix(y_true_group, y_pred_group)

    # Calculate Sensitivity (True Positive Rate)
    tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # Calculate Specificity (True Negative Rate)
    fpr = cm[0, 1] / (cm[0, 0] + cm[0, 1])

    # Calculate Equal Odds for the current group
    equal_odds_scores.append((tpr, fpr))
    # Add data to the table
    table.add_row([group, tpr, fpr])

  # Find the maximum absolute difference in TPR and FPR across groups
  max_diff = max(abs(tpr1 - tpr2) + abs(fpr1 - fpr2) for (tpr1, fpr1),
   (tpr2, fpr2) in zip(equal_odds_scores[::2], equal_odds_scores[1::2]))

#   print("Equalized Odds Score :",max_diff)
#   print(table)
  return max_diff



def fair_Eq_Opportunity(X,y_var,sensitive_var,prediction):
#   print("----------------------------------------------------")
#   print("\t\t\t",sensitive_var)
#   print('----------------------------------------------------')
  groups = X[sensitive_var].unique()
  equal_odds_scores = []

  # Create a PrettyTable object
  table = PrettyTable()
  table.field_names = ["Name", "TPR"]

  for group in groups:
    group_data = X[X[sensitive_var] == group]
    y_true_group = group_data[y_var]
    y_pred_group = group_data[prediction]

    cm = confusion_matrix(y_true_group, y_pred_group)

    # Calculate Sensitivity (True Positive Rate)
    tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # Calculate Specificity (True Negative Rate)
    fpr = cm[0, 1] / (cm[0, 0] + cm[0, 1])

    # Calculate Equal Odds for the current group
    equal_odds_scores.append((tpr, fpr))
    # Add data to the table
    table.add_row([group, tpr])

  # Find the maximum absolute difference in TPR and FPR across groups
  max_diff = max(abs(tpr1 - tpr2)  for (tpr1, fpr1),
   (tpr2, fpr2) in zip(equal_odds_scores[::2], equal_odds_scores[1::2]))

#   print("Equal Opportunity Score :",max_diff)
#   print(table)
  return max_diff

def fair_Treatment_Equality (X,y_var,sensitive_var,prediction):
#   print("----------------------------------------------------")
#   print("\t\t\t",sensitive_var)
#   print('----------------------------------------------------')
  groups = X[sensitive_var].unique()
  tpr_scores = []
  fpr_scores = []

  # Create a PrettyTable object
  table = PrettyTable()
  table.field_names = ["Name", "TPR",'FPR']

  for group in groups:
    group_data = X[X[sensitive_var] == group]
    y_true_group = group_data[y_var]
    y_pred_group = group_data[prediction]

    # Calculate True Positive Rate (Sensitivity)
    cm = confusion_matrix(y_true_group, y_pred_group)
    tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # Calculate False Positive Rate
    fpr = cm[0, 1] / (cm[0, 0] + cm[0, 1])

    # Append TPR and FPR for the current group
    tpr_scores.append(tpr)
    fpr_scores.append(fpr)

    table.add_row([group, tpr,fpr])

  # Find the maximum absolute difference in Proportion Positive across groups
  max_diff_tpr = max(abs(tpr1 - tpr2) for tpr1, tpr2 in combinations(tpr_scores, 2))
  max_diff_fpr = max(abs(fpr1 - fpr2) for fpr1, fpr2 in combinations(fpr_scores, 2))


#   print("Treatment Equality Score :",max(max_diff_tpr, max_diff_fpr))

#   print(table)
  return max(max_diff_tpr, max_diff_fpr)

# **Equal sampling function**

In [4]:
def equal_sampling(dataframe, variables):
    """
    Perform equal sampling from a pandas DataFrame based on a list of variables.

    Parameters:
    - dataframe: pandas DataFrame.
    - variables: list of column names in the dataframe to be used for creating groups.

    Returns:
    - A pandas DataFrame with equal representation from each group formed by the unique combinations of the specified variables.
    """
    # Generate all unique combinations of the specified variables
    combinations = dataframe.groupby(variables).size().reset_index().rename(columns={0: 'count'})

    # Determine the smallest group size among these combinations
    min_size = combinations['count'].min()

    # Initialize an empty DataFrame to store the sampled data
    sampled_df = pd.DataFrame(columns=dataframe.columns)

    # Loop through each unique combination, filter the dataframe, and sample
    for _, row in combinations.iterrows():
        filter_criteria = (dataframe[variables[0]] == row[variables[0]])
        for var in variables[1:]:
            filter_criteria &= (dataframe[var] == row[var])

        sampled_group = dataframe[filter_criteria].sample(n=min_size, replace=True) # Use replace=True if min_size is larger than the group
        sampled_df = pd.concat([sampled_df, sampled_group])

    return sampled_df

# **Import the dataset and read**

In [5]:
# # Initialize the OneHotEncoder
# encoder = OneHotEncoder(drop='first')  # drop='first' to avoid multicollinearity
# encoded_cols = encoder.fit_transform(df[categorical_cols_more_than_two]).toarray()  # Converts the sparse matrix output to a dense array

import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

def load_and_preprocess_adult_dataset():
    # Fetch the dataset
    data = fetch_openml(name='adult', version=2, as_frame=True)
    df = data.frame
    
    # Separate and store 'sex' column
    sex_df = df.loc[:,['sex']]
    
    # Separate the target column
    y_df = df.loc[:,['class']]
    
    # Drop the columns not needed for feature encoding
    df = df.drop(['sex', 'race', 'native-country', 'class'], axis=1)
    
    # Identify categorical columns with more than 2 unique values
    categorical_cols = df.select_dtypes(include=['category']).columns
    categorical_cols_more_than_two = [col for col in categorical_cols if len(df[col].unique()) > 2]
    
    # One-hot encode these columns
    encoder = OneHotEncoder(drop='first')  # Automatically handles multicollinearity by dropping the first category
    encoded_cols = encoder.fit_transform(df[categorical_cols_more_than_two]).toarray()
    
    # Create a DataFrame with the encoded columns
    encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols_more_than_two))
    
    # Drop original categorical columns from df and join with encoded_df
    df = df.drop(categorical_cols_more_than_two, axis=1)
    df = pd.concat([df, encoded_df], axis=1)
    
    return df, y_df, sex_df

# Example usage
df_adult, df_y, df_sen = load_and_preprocess_adult_dataset()

print(df_y.head())      # Print the first few rows of the target DataFrame
print(df_sen.head())      # Print the first few rows of the target DataFrame
print("DataFrame shape:", df_adult.shape)
initial_full_dataset = df_adult
initial_full_dataset


   class
0  <=50K
1  <=50K
2   >50K
3   >50K
4  <=50K
      sex
0    Male
1    Male
2    Male
3    Male
4  Female
DataFrame shape: (48842, 54)


,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,occupation_nan,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,226802,7,0,0,40,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,38,89814,9,0,0,50,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28,336951,12,0,0,40,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,44,160323,10,7688,0,40,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18,103497,10,0,0,30,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,257302,12,0,0,38,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
48838,40,154374,9,0,0,40,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48839,58,151910,9,0,0,40,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48840,22,201490,9,0,0,20,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
# url = "https://raw.githubusercontent.com/ImeshEkanayake/Fair_Datasets/main/Adult%20Census%20Income.csv"
# initial_full_dataset = pd.read_csv(url)
# initial_full_dataset

In [7]:
len(initial_full_dataset.columns)

54

# **Assign variable categories**

In [8]:
# X_var_con = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
# X_var_cat = ['workclass', 'marital-status',  'occupation','relationship','native-country']

# Sen_var = ['sex'] #'race',
# y_var = ['class']

# **One-Hot Encoding on Categorical variables**

In [9]:
df_sen.columns

Index(['sex'], dtype='object')

In [10]:

replacement_mapping = { '>50K': 1, '<=50K': 0}
df_y['class'] = df_y['class'].map(replacement_mapping).to_numpy()
df_y = pd.DataFrame(df_y)


replacement_mapping = { 'Male': 1, 'Female': 0}
df_sen['sex'] = df_sen['sex'].map(replacement_mapping).to_numpy()
df_sen = pd.DataFrame(df_sen)

# **Edit this cell acording to the varibale names**

In [11]:
ohe_full_dataset = df_adult
ohe_full_dataset["Sensitive"] = df_sen['sex']
ohe_full_dataset["Class"] = df_y['class']
df_X = df_adult
# median_value = ohe_full_dataset['Class_con'].median()
# ohe_full_dataset["Class"] = (ohe_full_dataset['Class_con'] >= median_value).astype(int)

In [12]:
df_results = pd.DataFrame(columns=['Data', 'Dataset_name',
        "Alpha-pred",
        "Beta-Fair",
        "Gamma-privacy",
         'Balance_Class', 'Balance_Sensitive',
       'Fairness_cost_function', 'Dataset_training_size',
       'Dataset_testing_size', 
        "Test_loss_predictor",
        "Test_loss_Discriminator_fool",
        "Test_total_loss_gen",
        "Train_loss_predictor",
        "Train_loss_Discriminator_fool",
        "Train_total_loss_gen",
       'Dataset_fairness_Demographic_Parity',
       'Report_DNN_Discriminator_Training_Accuracy',
       'Report_DNN_Discriminator_Training_precision',
       'Report_DNN_Discriminator_Training_recall',
       'Report_DNN_Discriminator_Training_f1',
       'Report_DNN_Discriminator_Testing_Accuracy',
       'Report_DNN_Discriminator_Testing_precision',
       'Report_DNN_Discriminator_Testing_recall',
       'Report_DNN_Discriminator_Testing_f1',
       'Report_Predictor_Training_Accuracy',
       'Report_Predictor_Training_precision',
       'Report_Predictor_Training_recall', 'Report_Predictor_Training_f1',
       'Report_Predictor_Testing_Accuracy',
       'Report_Predictor_Testing_precision', 'Report_Predictor_Testing_recall',
       'Report_Predictor_Testing_f1',
       'Report_Before_CatBoost_Prediction_Training_Accuracy',
       'Report_Before_CatBoost_Prediction_Training_precision',
       'Report_Before_CatBoost_Prediction_Training_recall',
       'Report_Before_CatBoost_Prediction_Training_f1',
       'Report_Before_CatBoost_Prediction_Testing_Accuracy',
       'Report_Before_CatBoost_Prediction_Testing_precision',
       'Report_Before_CatBoost_Prediction_Testing_recall',
       'Report_Before_CatBoost_Prediction_Testing_f1',
       'Report_After_CatBoost_Prediction_Training_Accuracy',
       'Report_After_CatBoost_Prediction_Training_precision',
       'Report_After_CatBoost_Prediction_Training_recall',
       'Report_After_CatBoost_Prediction_Training_f1',
       'Report_After_CatBoost_Prediction_Testing_Accuracy',
       'Report_After_CatBoost_Prediction_Testing_precision',
       'Report_After_CatBoost_Prediction_Testing_recall',
       'Report_After_CatBoost_Prediction_Testing_f1',
       'Report_Before_CatBoost_Sensitive_Training_Accuracy',
       'Report_Before_CatBoost_Sensitive_Training_precision',
       'Report_Before_CatBoost_Sensitive_Training_recall',
       'Report_Before_CatBoost_Sensitive_Training_f1',
       'Report_Before_CatBoost_Sensitive_Testing_Accuracy',
       'Report_Before_CatBoost_Sensitive_Testing_precision',
       'Report_Before_CatBoost_Sensitive_Testing_recall',
       'Report_Before_CatBoost_Sensitive_Testing_f1',
       'Report_After_CatBoost_Sensitive_Training_Accuracy',
       'Report_After_CatBoost_Sensitive_Training_precision',
       'Report_After_CatBoost_Sensitive_Training_recall',
       'Report_After_CatBoost_Sensitive_Training_f1',
       'Report_After_CatBoost_Sensitive_Testing_Accuracy',
       'Report_After_CatBoost_Sensitive_Testing_precision',
       'Report_After_CatBoost_Sensitive_Testing_recall',
       'Report_After_CatBoost_Sensitive_Testing_f1',
       'Fairness_Equal_Odd_Before_CatBoost_prediction_Training',
       'Fairness_Equal_Odd_Before_CatBoost_prediction_Testing',
       'Fairness_Equal_Odd_DNN_prediction_Training',
       'Fairness_Equal_Odd_DNN_prediction_Testing',
       'Fairness_Equal_Odd_After_CatBoost_prediction_Training',
       'Fairness_Equal_Odd_After_CatBoost_prediction_Testing',
       'Fairness_Equal_Opportunity_Before_CatBoost_prediction_Training',
       'Fairness_Equal_Opportunity_Before_CatBoost_prediction_Testing',
       'Fairness_Equal_Opportunity_DNN_prediction_Training',
       'Fairness_Equal_Opportunity_DNN_prediction_Testing',
       'Fairness_Equal_Opportunity_After_CatBoost_prediction_Training',
       'Fairness_Equal_Opportunity_After_CatBoost_prediction_Testing',
       'Fairness_Demographic_Parity_Before_CatBoost_prediction_Training',
       'Fairness_Demographic_Parity_Before_CatBoost_prediction_Testing',
       'Fairness_Demographic_Parity_DNN_prediction_Training',
       'Fairness_Demographic_Parity_DNN_prediction_Testing',
       'Fairness_Demographic_Parity_After_CatBoost_prediction_Training',
       'Fairness_Demographic_Parity_After_CatBoost_prediction_Testing',
       'Fairness_Treatment_Equality_Before_CatBoost_prediction_Training',
       'Fairness_Treatment_Equality_Before_CatBoost_prediction_Testing',
       'Fairness_Treatment_Equality_DNN_prediction_Training',
       'Fairness_Treatment_Equality_DNN_prediction_Testing',
       'Fairness_Treatment_Equality_After_CatBoost_prediction_Training',
       'Fairness_Treatment_Equality_After_CatBoost_prediction_Testing'])
df_results

,Data,Dataset_name,Alpha-pred,Beta-Fair,Gamma-privacy,Balance_Class,Balance_Sensitive,Fairness_cost_function,Dataset_training_size,Dataset_testing_size,...,Fairness_Demographic_Parity_DNN_prediction_Training,Fairness_Demographic_Parity_DNN_prediction_Testing,Fairness_Demographic_Parity_After_CatBoost_prediction_Training,Fairness_Demographic_Parity_After_CatBoost_prediction_Testing,Fairness_Treatment_Equality_Before_CatBoost_prediction_Training,Fairness_Treatment_Equality_Before_CatBoost_prediction_Testing,Fairness_Treatment_Equality_DNN_prediction_Training,Fairness_Treatment_Equality_DNN_prediction_Testing,Fairness_Treatment_Equality_After_CatBoost_prediction_Training,Fairness_Treatment_Equality_After_CatBoost_prediction_Testing


# **Bootstrap Loop**

In [13]:
from datetime import datetime
current_date = datetime.now()
Data = current_date.strftime("%Y-%m-%d %H:%M")

In [ ]:
import torch.autograd as autograd
import seaborn as sns
# Enable anomaly detection
def remove_duplicate_columns(df):
    df_transposed = df.T
    df_unique = df_transposed.drop_duplicates(keep='first').T
    return df_unique

#=========================Demographic Parity Loss for training purposes=============================
def demographic_parity_loss(predictor_pred, sen_batch):
    # Calculate the probabilities of positive outcome for each group
    positive_outcome_prob_group1 = predictor_pred[sen_batch == 0].mean()
    positive_outcome_prob_group2 = predictor_pred[sen_batch == 1].mean()
    # The loss is the absolute difference between these probabilities
    dp_loss = abs(positive_outcome_prob_group1 - positive_outcome_prob_group2)

    return dp_loss

#########################################################################################################################################

import numpy as np

def update_hyperparameters(alpha, beta, gamma, loss_gen_to_help_pred, loss_gen_to_fool_dis, dp_loss_predictor, learning_rate, alpha_m, alpha_v, beta_m, beta_v, gamma_m, gamma_v, t, beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Updates the hyperparameters alpha, beta, and gamma using the Adam optimizer based on the losses from the latest model training iteration.

    Parameters:
    alpha (float): Current value of alpha.
    beta (float): Current value of beta.
    gamma (float): Current value of gamma.
    loss_gen_to_help_pred (float): Latest loss associated with the predictor helping term.
    loss_gen_to_fool_dis (float): Latest loss associated with the discriminator fooling term.
    dp_loss_predictor (float): Latest loss associated with the fairness loss term for the predictor.
    learning_rate (float): Learning rate for the hyperparameter updates.
    alpha_m, alpha_v, beta_m, beta_v, gamma_m, gamma_v: Moving averages for Adam optimizer.
    t (int): Timestep for Adam optimizer.
    beta1, beta2 (float): Parameters for Adam optimizer.
    epsilon (float): Small value to prevent division by zero in Adam.

    # Initialize Adam parameters
    alpha_m, alpha_v, beta_m, beta_v, gamma_m, gamma_v, t = 0, 0, 0, 0, 0, 0, 0
    
    Returns:
    tuple: Updated values of (alpha, beta, gamma) and their Adam parameters.
    """
    # Calculate gradients based on inverse loss proportion
    total_loss = loss_gen_to_help_pred + loss_gen_to_fool_dis + dp_loss_predictor
    grad_alpha = -(loss_gen_to_help_pred / total_loss)
    grad_beta = -(dp_loss_predictor / total_loss)
    grad_gamma = -(loss_gen_to_fool_dis / total_loss)

    # Update hyperparameters using Adam
    def adam_update(param, grad, m, v, t):
        t += 1
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * (grad ** 2)
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        param -= learning_rate * m_hat / (np.sqrt(v_hat.detach().numpy()) + epsilon)
        return param, m, v, t

    alpha, alpha_m, alpha_v, t = adam_update(alpha, grad_alpha, alpha_m, alpha_v, t)
    beta, beta_m, beta_v, t = adam_update(beta, grad_beta, beta_m, beta_v, t)
    gamma, gamma_m, gamma_v, t = adam_update(gamma, grad_gamma, gamma_m, gamma_v, t)

    return (alpha, beta, gamma, alpha_m, alpha_v, beta_m, beta_v, gamma_m, gamma_v, t)

#########################################################################################################################################

autograd.set_detect_anomaly(True)
Dataset_name = "German_Credit"
ds_count =0
Balance_Sensitive = 1
Balance_Class = 0


ds_count+=1
#------------------Balancing the dataset------------------------------------------------------------
balanced_full_dataset = ohe_full_dataset.copy()

if Balance_Sensitive==1 and Balance_Class==1:
    balanced_full_dataset = equal_sampling(ohe_full_dataset, ['Sensitive','Class'])
elif Balance_Sensitive==1 :
    balanced_full_dataset = equal_sampling(ohe_full_dataset, ['Sensitive'])
elif Balance_Class==1 :
    balanced_full_dataset = equal_sampling(ohe_full_dataset, ['Class'])
else:
    balanced_full_dataset = ohe_full_dataset.loc[:,list(df_X.columns)+["Sensitive","Class"]]

balanced_full_dataset = balanced_full_dataset.loc[:,list(df_X.columns)+["Sensitive","Class"]]
balanced_full_dataset = balanced_full_dataset.dropna()
balanced_full_dataset = remove_duplicate_columns(balanced_full_dataset)
Dataset_fairness_Demographic_Parity = fair_Demographic_Parity(balanced_full_dataset,y_var='Class',sensitive_var='Sensitive',prediction='Class')

# Selecting specific variables and dropping NA
X_var = list(df_X.columns)
Sen_var = df_sen.columns
y_var = df_y.columns
for i in X_var:
    if i not in balanced_full_dataset.columns:
        X_var.remove(i)

# Splitting the dataset
# X = balanced_full_dataset.loc[:, list(df_X.columns)].values
X = balanced_full_dataset
X.drop(['Class',"Sensitive"], axis=1)
y = balanced_full_dataset.loc[:,  "Class"].values
sen = balanced_full_dataset.loc[:, ["Sensitive"]].values
X_var = list(X.columns)
#trainspose the dataset
scaler = StandardScaler()
scaler.fit(X)
XT = scaler.transform(X)
random_int = random.randint(0, 10000)
# Assuming X, y, and z are your datasets
X_train, X_test, y_train, y_test,sen_train, sen_test = train_test_split(XT, y,sen, test_size=0.3, random_state=random_int)
X_train, X_test, y_train, y_test,sen_train, sen_test = X_train.astype(np.float32), X_test.astype(np.float32), y_train.astype(np.float32), y_test.astype(np.float32),sen_train.astype(np.float32), sen_test.astype(np.float32)
y_train, y_test,sen_train, sen_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1),sen_train.reshape(-1, 1), sen_test.reshape(-1, 1)

Dataset_training_size = len(y_train)
Dataset_testing_size = len(y_test)

count=0
Alpha = 0.33
Beta = 0.33
Gamma = 1-Beta-Alpha
time=[]
while count<50:
    count+=1
    print("Dataset_count",ds_count,"count:",str(count),"iter:",iter,"Sensitive:",
          Balance_Sensitive,"Class:",Balance_Class,' a:'+str(Alpha)+' b:'+str(Beta)+' r:'+str(Gamma))

    #=================================================== Model ====================================================
    #########################################
    # Attention Module (Feature-Attention)
    #########################################
    class FeatureAttention(nn.Module):
        def __init__(self, dim, reduction=4):
            super(FeatureAttention, self).__init__()
            self.fc1 = nn.Linear(dim, dim // reduction)
            self.fc2 = nn.Linear(dim // reduction, dim)
            self.sigmoid = nn.Sigmoid()
            
        def forward(self, x):
            # x shape: (batch_size, dim)
            att = self.fc1(x)
            att = F.leaky_relu(att, 0.2)
            att = self.fc2(att)
            att = self.sigmoid(att)
            return x * att
    
    #########################################
    # Generator with Increased Depth, Neurons,
    # Normalization, Dropout, Residual Connection, and Attention
    #########################################
    class Generator(nn.Module):
        def __init__(self):
            super(Generator, self).__init__()
            # Expanded layers with increased neurons
            self.gen1 = nn.Linear(X_train.shape[1], 64)
            self.ln1 = nn.LayerNorm(64)
            
            self.gen2 = nn.Linear(64, 128)
            self.ln2 = nn.LayerNorm(128)
            
            self.gen3 = nn.Linear(128, 256)
            self.ln3 = nn.LayerNorm(256)
            
            self.gen4 = nn.Linear(256, 256)
            self.ln4 = nn.LayerNorm(256)
            
            self.gen5 = nn.Linear(256, 128)
            self.ln5 = nn.LayerNorm(128)
            
            self.gen6 = nn.Linear(128, 64)
            self.ln6 = nn.LayerNorm(64)
            
            self.gen7 = nn.Linear(64, X_train.shape[1])
            
            # Attention layer is applied after the last hidden block
            self.attn = FeatureAttention(64)
            
            # Dropout for regularization
            self.dropout = nn.Dropout(0.1)
        
        def forward(self, x):
            x = F.leaky_relu(self.ln1(self.gen1(x)), 0.2)
            residual = x  # Save for residual connection (64-dim)
            
            x = F.leaky_relu(self.ln2(self.gen2(x)), 0.2)
            x = self.dropout(x)
            x = F.leaky_relu(self.ln3(self.gen3(x)), 0.2)
            x = self.dropout(x)
            x = F.leaky_relu(self.ln4(self.gen4(x)), 0.2)
            x = self.dropout(x)
            x = F.leaky_relu(self.ln5(self.gen5(x)), 0.2)
            x = self.dropout(x)
            x = F.leaky_relu(self.ln6(self.gen6(x)), 0.2)
            x = self.dropout(x)
            
            # Residual connection: add features from the first layer
            x = x + residual
            
            # Apply attention to recalibrate features
            x = self.attn(x)
            
            return torch.sigmoid(self.gen7(x))
    class Discriminator(nn.Module):
        def __init__(self):
            super(Discriminator, self).__init__()
            # Discriminator
            self.dis1 = nn.Linear(X_train.shape[1], 32)
            self.dis2 = nn.Linear(32, 64)
            self.dis3 = nn.Linear(64, 32)
            self.dis4 = nn.Linear(32, 1)

        def forward(self, x):
            x = F.leaky_relu(self.dis1(x), 0.1)
            x = F.leaky_relu(self.dis2(x), 0.1)
            x = F.leaky_relu(self.dis3(x), 0.1)
            discriminator_output = F.sigmoid(self.dis4(x))
            return discriminator_output

    class Predictor(nn.Module):
        def __init__(self):
            super(Predictor, self).__init__()
            # Predictor
            self.pred1 = nn.Linear(X_train.shape[1], 32)
            self.pred2 = nn.Linear(32, 64)
            self.pred3 = nn.Linear(64, 128)
            self.pred4 = nn.Linear(128, 64)
            self.pred5 = nn.Linear(64, 32)
            self.pred6 = nn.Linear(32, 1)

        def forward(self, x):
            x = F.leaky_relu(self.pred1(x), 0.3)
            x = F.leaky_relu(self.pred2(x), 0.3)
            x = F.leaky_relu(self.pred3(x), 0.3)
            x = F.leaky_relu(self.pred4(x), 0.3)
            x = F.leaky_relu(self.pred5(x), 0.3)
            prediction_output = F.sigmoid(self.pred6(x))

            return prediction_output

    batch_size = 128

    #================== Create the training and testing dataloaders Train ==============================

    # Converting to PyTorch tensors
    X_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_tensor = torch.tensor(y_train, dtype=torch.float32)
    sen_tensor = torch.tensor(sen_train, dtype=torch.float32)

    # Creating a TensorDataset
    dataset = TensorDataset(X_tensor, y_tensor, sen_tensor)

    # Create a DataLoader for batch processing
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


    #================== Create the training and testing dataloaders Test ================================

    # Converting to PyTorch tensors
    X_Test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_Test_tensor = torch.tensor(y_test, dtype=torch.float32)
    sen_Test_tensor = torch.tensor(sen_test, dtype=torch.float32)

    # Creating a TensorDataset
    dataset_Test = TensorDataset(X_Test_tensor, y_Test_tensor, sen_Test_tensor)

    # Create a DataLoader for batch processing
    Test_data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    

    #=============================================================================================
    #               Optimizer Expected values for accuracy, fairness and privacy
    #==============================================================================================
    from sklearn.metrics import classification_report
    from catboost import CatBoostClassifier
    
    Before_catboost_model_class = CatBoostClassifier(border_count= 78, iterations= 30,
                                        l2_leaf_reg= 1, learning_rate= 0.2, min_data_in_leaf= 15,
                                        loss_function='CrossEntropy',verbose=0)

    Before_catboost_model_class.fit(X_tensor.detach().numpy(), y_tensor.numpy())
    class_labels = Before_catboost_model_class.predict(X_Test_tensor.detach().numpy())
    report = classification_report(y_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Expected_predictor_accuracy =  df_report['precision']['accuracy']
    Expected_predictor_fairness = 0.01
    Expected_discriminator_accuracy = 0.50

    #================================Model, loss function and optimizer Initiation======================
    # Instantiate models
    generator = Generator()
    discriminator = Discriminator()
    predictor = Predictor()

    # Define loss functions and optimizers
    criterion_fake_real = nn.BCELoss()
    criterion_classification = nn.BCELoss()
    criterion_prediction = nn.BCELoss()

    optimizer_gen = optim.Adam(generator.parameters(), lr=0.001)
    optimizer_dis = optim.Adam(discriminator.parameters(), lr=0.001)
    optimizer_pred = optim.Adam(predictor.parameters(), lr=0.002)


    ####################################################################################################
    #                                   Training Loop
    ####################################################################################################
    num_epochs = 50  # Number of epochs

    start_time = datetime.now()
    Date = current_date.strftime("%Y-%m-%d %H:%M")

    for epoch in range(num_epochs):
        if epoch%10==0:
            print("|",end="")
        else:
            print(".",end="")
        for batch in data_loader:
            # Load batch data
            X_batch, y_batch, sen_batch = batch

            # Reset gradients for all optimizers
            optimizer_gen.zero_grad()
            optimizer_dis.zero_grad()
            optimizer_pred.zero_grad()

            # Forward pass through generator
            gen_data = generator(X_batch)

            # Discriminator Training
            # Discriminator tries to predict sensitive attribute from generated data
            discriminator_pred = discriminator(gen_data)
            loss_discriminator = criterion_fake_real(discriminator_pred, sen_batch)
            loss_discriminator.backward(retain_graph=True)  # Retain graph for generator's update
            optimizer_dis.step()

            # Predictor Training
            # Predictor tries to predict target from generated data
            predictor_pred = predictor(gen_data)
            dp_loss_predictor = demographic_parity_loss(predictor_pred, sen_batch)
            loss_predictor = criterion_prediction(predictor_pred, y_batch) + (dp_loss_predictor* Beta)
            loss_predictor.backward(retain_graph=True)  # Retain graph for generator's update
            optimizer_pred.step()

            # Generator Training
            discriminator_pred_for_gen = discriminator(gen_data)
            loss_gen_to_fool_dis = criterion_fake_real(discriminator_pred_for_gen, 1 - sen_batch)# Generator tries to fool discriminator and help predictor
            disc_accuracy = accuracy_score(np.round(discriminator_pred_for_gen.detach().numpy()), sen_batch)# Fooling discriminator

            # Helping predictor
            predictor_pred_for_gen = predictor(gen_data)
            loss_gen_to_help_pred = criterion_classification(predictor_pred_for_gen, y_batch)
            pred_accuracy = accuracy_score(np.round(predictor_pred_for_gen.detach().numpy()), y_batch)

            # Combine losses for generator
            dp_loss_predictor = demographic_parity_loss(predictor(generator(X_batch)), sen_batch)
            total_loss_gen =   Alpha*loss_gen_to_help_pred*5 + Gamma*loss_gen_to_fool_dis + Beta*dp_loss_predictor
            total_loss_gen.backward()
            optimizer_gen.step()


    Test_class_pred = np.where(predictor( generator(X_Test_tensor)) >= 0.5, 1, 0)
    Test_sen_pred = np.where(predictor( generator(X_Test_tensor)) >= 0.5, 1, 0)
    Test_pred_accuracy = accuracy_score(Test_class_pred, y_Test_tensor.numpy())
    Test_dp_loss_predictor = demographic_parity_loss(Test_sen_pred, sen_Test_tensor.numpy())
    Test_discriminator_pred = np.where(discriminator( generator(X_Test_tensor))>= 0.5, 1, 0)
    Test_discriminator_accuracy = accuracy_score(Test_discriminator_pred, sen_Test_tensor.numpy())

    Test_discriminator_pred_for_gen = discriminator(generator(X_Test_tensor))
    Test_loss_gen_to_fool_dis = criterion_fake_real(Test_discriminator_pred_for_gen, 1 - sen_Test_tensor)
    Test_loss_gen_to_help_pred = criterion_classification(predictor( generator(X_Test_tensor)), y_Test_tensor)
    Test_dp_loss_predictor = demographic_parity_loss(predictor(generator(X_Test_tensor)), sen_Test_tensor)
    Test_total_loss_gen =   Test_loss_gen_to_help_pred + Test_loss_gen_to_fool_dis + Test_dp_loss_predictor

    Train_class_pred = np.where(predictor( generator(X_tensor)) >= 0.5, 1, 0)
    Train_sen_pred = np.where(predictor( generator(X_tensor)) >= 0.5, 1, 0)
    Train_pred_accuracy = accuracy_score(Train_class_pred, y_tensor.numpy())
    Train_dp_loss_predictor = demographic_parity_loss(Train_sen_pred, sen_tensor.numpy())
    Train_discriminator_pred = np.where(discriminator( generator(X_tensor))>= 0.5, 1, 0)
    Train_discriminator_accuracy = accuracy_score(Train_discriminator_pred, sen_tensor.numpy())

    Train_discriminator_pred_for_gen = discriminator(generator(X_tensor))
    Train_loss_gen_to_fool_dis = criterion_fake_real(Train_discriminator_pred_for_gen, 1 - sen_tensor)
    Train_loss_gen_to_help_pred = criterion_classification(predictor( generator(X_tensor)), y_tensor)
    Train_dp_loss_predictor = demographic_parity_loss(predictor(generator(X_batch)), sen_batch)
    Train_total_loss_gen =   Train_loss_gen_to_help_pred + Train_loss_gen_to_fool_dis + Train_dp_loss_predictor

    current_time = datetime.now()
    delta = current_time - start_time
    time+=[delta]
    

    #=================================================================
    #              Plot the prediction probabilities
    #=================================================================
    class_labels = np.where(predictor( generator(X_Test_tensor)) >= 0.5, 1, 0)
    report = classification_report(y_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_Predictor_Testing_Accuracy =  df_report['precision']['accuracy']
    #---------------------------------------------------------------------------------------------------------------------------
    class_labels = np.where(discriminator( generator(X_Test_tensor)) >= 0.5, 1, 0)
    report = classification_report(sen_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_DNN_Discriminator_Testing_Accuracy =  df_report['precision']['accuracy']
    #---------------------------------------------------------------------------------------------------------------------------
    catboost_model = CatBoostClassifier(border_count= 78, depth= 5, iterations= 20,
                                                            l2_leaf_reg= 3, learning_rate= 0.2, min_data_in_leaf= 15,
                                                            loss_function='CrossEntropy',verbose=0)
    catboost_model.fit(generator(X_tensor).detach().numpy(), sen_tensor.numpy())
    class_labels = catboost_model.predict(generator(X_Test_tensor).detach().numpy())
    report = classification_report(sen_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_After_CatBoost_Sensitive_Testing_Accuracy =  df_report['precision']['accuracy']
    #---------------------------------------------------------------------------------------------------------------------------
    
    probabilities_class = np.array(predictor(generator(X_Test_tensor)).detach())[:,0]
    probabilities_sex = np.array(discriminator(generator(X_Test_tensor)).detach())[:,0]
    class_labels = catboost_model.predict_proba(generator(X_Test_tensor).detach().numpy())
    probabilities_sex_catB = np.array(class_labels[:,1])
    
    # Define the bin width
    bin_width = 0.02
    
    # Determine the global minimum and maximum to ensure all histograms use the same bins
    min_prob = 0
    max_prob = 1
    
    # Create bins from the minimum to the maximum value with the specified bin width
    bins = np.arange(min_prob, max_prob + bin_width, bin_width)
    
    # Format accuracy values to three decimal points
    formatted_class_accuracy = f"{Report_Predictor_Testing_Accuracy:.3f}"
    formatted_gender_accuracy = f"{Report_DNN_Discriminator_Testing_Accuracy:.3f}"
    formatted_gender_catb_accuracy = f"{Report_After_CatBoost_Sensitive_Testing_Accuracy:.3f}"
    
    # Plot histograms with formatted labels
    plt.hist(probabilities_class, bins=bins, density=True, alpha=0.5, label=f'Class (Acc: {formatted_class_accuracy})')
    plt.hist(probabilities_sex, bins=bins, density=True, alpha=0.5, label=f'Gender (Acc: {formatted_gender_accuracy})')
    plt.hist(probabilities_sex_catB, bins=bins, density=True, alpha=0.5, label=f'Gender_CatB (Acc: {formatted_gender_catb_accuracy})')
    
    plt.legend(title='Probability Distributions')
    plt.title('Density Plot of Prediction Probabilities at CM: '+' a:'+str(Alpha)+' b:'+str(Beta)+' r:'+str(Gamma))
    plt.xlabel('Probability')
    plt.ylabel('Density')
    plt.yscale('log')  # Set the y-axis to logarithmic scale
    plt.savefig('Plots_op_2/Density Plot of Prediction Probabilities at CM: ' + ' a:'+str(np.round(Alpha,2))+' b:'+str(np.round(Beta,2))+' r:'+str(np.round(Gamma,2))+'.png')
    plt.show()

    from sklearn.metrics import log_loss
    from catboost import CatBoostClassifier
    catboost_model = CatBoostClassifier(border_count= 78, depth= 5, iterations= 20,
                                        l2_leaf_reg= 3, learning_rate= 0.2, min_data_in_leaf= 15,
                                        loss_function='CrossEntropy',verbose=0)

    catboost_model.fit(generator(X_tensor).detach().numpy(), sen_tensor.numpy())
    class_labels = catboost_model.predict(generator(X_tensor).detach().numpy())
    class_labels = 1 - class_labels
    cat_sen_bce_loss = log_loss(sen_tensor.numpy(), class_labels)
    print('------------cat_dis_loss',cat_sen_bce_loss)

    
    


    #=================================================================
    #               Discriminator report Training
    #=================================================================
    from sklearn.metrics import classification_report
    class_labels = np.where(discriminator( generator(X_tensor)) >= 0.5, 1, 0)
    report = classification_report(sen_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_DNN_Discriminator_Training_Accuracy =  df_report['precision']['accuracy']
    Report_DNN_Discriminator_Training_precision = df_report['precision']['weighted avg']
    Report_DNN_Discriminator_Training_recall = df_report['recall']['weighted avg']
    Report_DNN_Discriminator_Training_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               Discriminator report Tessting
    #=================================================================
    from sklearn.metrics import classification_report
    class_labels = np.where(discriminator( generator(X_Test_tensor)) >= 0.5, 1, 0)
    report = classification_report(sen_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_DNN_Discriminator_Testing_Accuracy =  df_report['precision']['accuracy']
    Report_DNN_Discriminator_Testing_precision = df_report['precision']['weighted avg']
    Report_DNN_Discriminator_Testing_recall = df_report['recall']['weighted avg']
    Report_DNN_Discriminator_Testing_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               Predictor report Training
    #=================================================================
    from sklearn.metrics import classification_report
    class_labels = np.where(predictor( generator(X_tensor)) >= 0.5, 1, 0)
    report = classification_report(y_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_Predictor_Training_Accuracy =  df_report['precision']['accuracy']
    Report_Predictor_Training_precision = df_report['precision']['weighted avg']
    Report_Predictor_Training_recall = df_report['recall']['weighted avg']
    Report_Predictor_Training_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               Predictor report Testing
    #=================================================================
    from sklearn.metrics import classification_report
    class_labels = np.where(predictor( generator(X_Test_tensor)) >= 0.5, 1, 0)
    report = classification_report(y_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_Predictor_Testing_Accuracy =  df_report['precision']['accuracy']
    Report_Predictor_Testing_precision = df_report['precision']['weighted avg']
    Report_Predictor_Testing_recall = df_report['recall']['weighted avg']
    Report_Predictor_Testing_f1 = df_report['f1-score']['weighted avg']


    #=================================================================
    #               Before CatBoost Predictor report - Training
    #=================================================================
    from sklearn.metrics import classification_report

    from catboost import CatBoostClassifier
    Before_catboost_model_class = CatBoostClassifier(border_count= 78, depth= 5, iterations= 20,
                                        l2_leaf_reg= 3, learning_rate= 0.2, min_data_in_leaf= 15,
                                        loss_function='CrossEntropy',verbose=0)

    Before_catboost_model_class.fit(X_tensor.detach().numpy(), y_tensor.numpy())
    print("Training Score:",Before_catboost_model_class.score(X_tensor.detach().numpy(), y_tensor.numpy()))

    class_labels = Before_catboost_model_class.predict(X_tensor.detach().numpy())
    report = classification_report(y_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_Before_CatBoost_Prediction_Training_Accuracy =  df_report['precision']['accuracy']
    Report_Before_CatBoost_Prediction_Training_precision = df_report['precision']['weighted avg']
    Report_Before_CatBoost_Prediction_Training_recall = df_report['recall']['weighted avg']
    Report_Before_CatBoost_Prediction_Training_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               Before CatBoost Predictor report - Testing
    #=================================================================
    class_labels = Before_catboost_model_class.predict(X_Test_tensor.detach().numpy())
    report = classification_report(y_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_Before_CatBoost_Prediction_Testing_Accuracy =  df_report['precision']['accuracy']
    Report_Before_CatBoost_Prediction_Testing_precision = df_report['precision']['weighted avg']
    Report_Before_CatBoost_Prediction_Testing_recall = df_report['recall']['weighted avg']
    Report_Before_CatBoost_Prediction_Testing_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               After CatBoost Predictor report
    #=================================================================
    from catboost import CatBoostClassifier
    After_catboost_model_class = CatBoostClassifier(border_count= 178, depth= 8, iterations= 20,
                                        l2_leaf_reg= 1, learning_rate= 0.3, min_data_in_leaf= 5,
                                        loss_function='CrossEntropy',verbose=0 )

    After_catboost_model_class.fit(generator(X_tensor).detach().numpy(), y_tensor.numpy())
    print("Training Score:",After_catboost_model_class.score(generator(X_tensor).detach().numpy(), y_tensor.numpy()))

    class_labels = After_catboost_model_class.predict(generator(X_tensor).detach().numpy())
    report = classification_report(y_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_After_CatBoost_Prediction_Training_Accuracy =  df_report['precision']['accuracy']
    Report_After_CatBoost_Prediction_Training_precision = df_report['precision']['weighted avg']
    Report_After_CatBoost_Prediction_Training_recall = df_report['recall']['weighted avg']
    Report_After_CatBoost_Prediction_Training_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               Before CatBoost Predictor report - Testing
    #=================================================================
    class_labels = After_catboost_model_class.predict(generator(X_Test_tensor).detach().numpy())
    report = classification_report(y_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_After_CatBoost_Prediction_Testing_Accuracy =  df_report['precision']['accuracy']
    Report_After_CatBoost_Prediction_Testing_precision = df_report['precision']['weighted avg']
    Report_After_CatBoost_Prediction_Testing_recall = df_report['recall']['weighted avg']
    Report_After_CatBoost_Prediction_Testing_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #            Before CatBoost Sen_Var Predictor report - Training
    #=================================================================
    from catboost import CatBoostClassifier
    catboost_model = CatBoostClassifier(border_count= 78, depth= 5, iterations= 20,
                                        l2_leaf_reg= 3, learning_rate= 0.2, min_data_in_leaf= 15,
                                        loss_function='CrossEntropy',verbose=0)

    catboost_model.fit(X_tensor.detach().numpy(), sen_tensor.numpy())
    print("Training Score:",catboost_model.score(X_tensor.detach().numpy(), sen_tensor.numpy()))

    class_labels = catboost_model.predict(X_tensor.detach().numpy())
    report = classification_report(sen_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_Before_CatBoost_Sensitive_Training_Accuracy =  df_report['precision']['accuracy']
    Report_Before_CatBoost_Sensitive_Training_precision = df_report['precision']['weighted avg']
    Report_Before_CatBoost_Sensitive_Training_recall = df_report['recall']['weighted avg']
    Report_Before_CatBoost_Sensitive_Training_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               Before CatBoost Sen_Var Predictor report - Testing
    #=================================================================
    class_labels = catboost_model.predict(X_Test_tensor.detach().numpy())
    report = classification_report(sen_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_Before_CatBoost_Sensitive_Testing_Accuracy =  df_report['precision']['accuracy']
    Report_Before_CatBoost_Sensitive_Testing_precision = df_report['precision']['weighted avg']
    Report_Before_CatBoost_Sensitive_Testing_recall = df_report['recall']['weighted avg']
    Report_Before_CatBoost_Sensitive_Testing_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #            After CatBoost Sen_Var Predictor report - Training
    #=================================================================
    from catboost import CatBoostClassifier
    catboost_model = CatBoostClassifier(border_count= 78, depth= 5, iterations= 20,
                                        l2_leaf_reg= 3, learning_rate= 0.2, min_data_in_leaf= 15,
                                        loss_function='CrossEntropy',verbose=0)

    catboost_model.fit(generator(X_tensor).detach().numpy(), sen_tensor.numpy())
    print("Training Score:",catboost_model.score(generator(X_tensor).detach().numpy(), sen_tensor.numpy()))

    class_labels = catboost_model.predict(generator(X_tensor).detach().numpy())
    report = classification_report(sen_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_After_CatBoost_Sensitive_Training_Accuracy =  df_report['precision']['accuracy']
    Report_After_CatBoost_Sensitive_Training_precision = df_report['precision']['weighted avg']
    Report_After_CatBoost_Sensitive_Training_recall = df_report['recall']['weighted avg']
    Report_After_CatBoost_Sensitive_Training_f1 = df_report['f1-score']['weighted avg']

    #=================================================================
    #               After CatBoost Sen_Var Predictor report - Testing
    #=================================================================
    class_labels = catboost_model.predict(generator(X_Test_tensor).detach().numpy())
    report = classification_report(sen_Test_tensor.numpy(), class_labels, target_names=['Class 0', 'Class 1'],output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    Report_After_CatBoost_Sensitive_Testing_Accuracy =  df_report['precision']['accuracy']
    Report_After_CatBoost_Sensitive_Testing_precision = df_report['precision']['weighted avg']
    Report_After_CatBoost_Sensitive_Testing_recall = df_report['recall']['weighted avg']
    Report_After_CatBoost_Sensitive_Testing_f1 = df_report['f1-score']['weighted avg']


    #=================================================================
    #               Merge the data to a single data-frame
    #=================================================================
    Training_df = pd.DataFrame(X_train,columns=X_var)
    Training_df["sen"] = sen_train
    Training_df["pred"] = y_train

    Training_df["After_prediction_DNN"] = np.where(predictor( generator(X_tensor)) >= 0.5, 1, 0)
    Training_df["After_prediction_CatB"] = After_catboost_model_class.predict(generator(X_tensor).detach().numpy())
    Training_df["Before_prediction_CatB"] = Before_catboost_model_class.predict(X_tensor.detach().numpy())


    Testing_df = pd.DataFrame(X_test,columns=X_var)
    Testing_df["sen"] = sen_test
    Testing_df["pred"] = y_test

    Testing_df["After_prediction_DNN"] = np.where(predictor( generator(X_Test_tensor)) >= 0.5, 1, 0)
    Testing_df["After_prediction_CatB"] = After_catboost_model_class.predict(generator(X_Test_tensor).detach().numpy())
    Testing_df["Before_prediction_CatB"] = Before_catboost_model_class.predict(X_Test_tensor.detach().numpy())

    #=================================================================
    #              Plot the prediction probabilities
    #=================================================================
    probabilities_class = np.array(predictor(generator(X_Test_tensor)).detach())[:,0]
    probabilities_sex = np.array(discriminator(generator(X_Test_tensor)).detach())[:,0]
    class_labels = catboost_model.predict_proba(generator(X_Test_tensor).detach().numpy())
    probabilities_sex_catB = np.array(class_labels[:,1])
    
    # Define the bin width
    bin_width = 0.02
    
    # Determine the global minimum and maximum to ensure all histograms use the same bins
    min_prob = min(probabilities_class.min(), probabilities_sex.min(), probabilities_sex_catB.min())
    max_prob = max(probabilities_class.max(), probabilities_sex.max(), probabilities_sex_catB.max())
    
    # Create bins from the minimum to the maximum value with the specified bin width
    bins = np.arange(min_prob, max_prob + bin_width, bin_width)
    
    # Format accuracy values to three decimal points
    formatted_class_accuracy = f"{Report_Predictor_Testing_Accuracy:.3f}"
    formatted_gender_accuracy = f"{Report_DNN_Discriminator_Testing_Accuracy:.3f}"
    formatted_gender_catb_accuracy = f"{Report_After_CatBoost_Sensitive_Testing_Accuracy:.3f}"
    
    # Plot histograms with formatted labels
    plt.hist(probabilities_class, bins=bins, density=True, alpha=0.5, label=f'Class (Acc: {formatted_class_accuracy})')
    plt.hist(probabilities_sex, bins=bins, density=True, alpha=0.5, label=f'Gender (Acc: {formatted_gender_accuracy})')
    plt.hist(probabilities_sex_catB, bins=bins, density=True, alpha=0.5, label=f'Gender_CatB (Acc: {formatted_gender_catb_accuracy})')
    
    plt.legend(title='Probability Distributions')
    plt.title('Density Plot of Prediction Probabilities at CM: ' + ' a:'+str(Alpha)+' b:'+str(Beta)+' r:'+str(Gamma))
    plt.xlabel('Probability')
    plt.ylabel('Density')
    # plt.yscale('log')  # Set the y-axis to logarithmic scale
    plt.savefig('plots_op_4_GC/Density Plot of Prediction Probabilities at CM: ' + ' a:'+str(np.round(Alpha,2))+' b:'+str(np.round(Beta,2))+' r:'+str(np.round(Gamma,2))+'.png')
    plt.show()


    #=================================================================
    #                  Equal Odd
    #=================================================================
    Fairness_Equal_Odd_Before_CatBoost_prediction_Training = fair_Eq_Odds(Training_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Equal_Odd_Before_CatBoost_prediction_Testing = fair_Eq_Odds(Testing_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Equal_Odd_DNN_prediction_Training = fair_Eq_Odds(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')
    Fairness_Equal_Odd_DNN_prediction_Testing = fair_Eq_Odds(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')
    Fairness_Equal_Odd_After_CatBoost_prediction_Training = fair_Eq_Odds(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')
    Fairness_Equal_Odd_After_CatBoost_prediction_Testing = fair_Eq_Odds(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')

    #=================================================================
    #                  Equal Opportunity
    #=================================================================
    Fairness_Equal_Opportunity_Before_CatBoost_prediction_Training = fair_Eq_Opportunity(Training_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Equal_Opportunity_Before_CatBoost_prediction_Testing = fair_Eq_Opportunity(Testing_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Equal_Opportunity_DNN_prediction_Training = fair_Eq_Opportunity(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_DNN')
    Fairness_Equal_Opportunity_DNN_prediction_Testing = fair_Eq_Opportunity(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_DNN')
    Fairness_Equal_Opportunity_After_CatBoost_prediction_Training = fair_Eq_Opportunity(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')
    Fairness_Equal_Opportunity_After_CatBoost_prediction_Testing = fair_Eq_Opportunity(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')

    #=================================================================
    #                  Demographic Parity
    #=================================================================
    Fairness_Demographic_Parity_Before_CatBoost_prediction_Training = fair_Demographic_Parity(Training_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Demographic_Parity_Before_CatBoost_prediction_Testing = fair_Demographic_Parity(Testing_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Demographic_Parity_DNN_prediction_Training = fair_Demographic_Parity(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_DNN')
    Fairness_Demographic_Parity_DNN_prediction_Testing = fair_Demographic_Parity(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_DNN')
    Fairness_Demographic_Parity_After_CatBoost_prediction_Training = fair_Demographic_Parity(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')
    Fairness_Demographic_Parity_After_CatBoost_prediction_Testing = fair_Demographic_Parity(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')

    #=================================================================
    #                  Treatment Equality
    #=================================================================
    Fairness_Treatment_Equality_Before_CatBoost_prediction_Training = fair_Treatment_Equality(Training_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Treatment_Equality_Before_CatBoost_prediction_Testing = fair_Treatment_Equality(Testing_df,y_var='pred',sensitive_var='sen',prediction='Before_prediction_CatB')
    Fairness_Treatment_Equality_DNN_prediction_Training = fair_Treatment_Equality(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_DNN')
    Fairness_Treatment_Equality_DNN_prediction_Testing = fair_Treatment_Equality(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_DNN')
    Fairness_Treatment_Equality_After_CatBoost_prediction_Training = fair_Treatment_Equality(Training_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')
    Fairness_Treatment_Equality_After_CatBoost_prediction_Testing = fair_Treatment_Equality(Testing_df,y_var='pred',sensitive_var='sen',prediction='After_prediction_CatB')

    #=================================================================
    #                  create the results table
    #=================================================================

    current_date = datetime.now()
    Date = current_date.strftime("%Y-%m-%d %H:%M")


    results = { "Data": [Date],
    "Dataset_name": [Dataset_name],
    "Alpha-pred": [Alpha],
    "Beta-Fair": [Beta],
    "Gamma-privacy": [Gamma],
    # "cost_multiplier-all": [cost_multiplier],
    "Balance_Class": [Balance_Class] ,
    "Balance_Sensitive": [Balance_Sensitive],
    "Dataset_training_size": [Dataset_training_size],
    "Dataset_testing_size": [Dataset_testing_size],
    # "Fairness_cost_function": [Fairness_cost_function],
    # "Fairness_cost_multiplier": [Fairness_cost_multiplier],
    "Test_loss_predictor":[Test_loss_gen_to_help_pred.detach().numpy()],
    "Test_loss_Discriminator_fool": [Test_loss_gen_to_fool_dis.detach().numpy()],
    "Test_total_loss_gen": [Test_total_loss_gen.detach().numpy()],
    "Train_loss_predictor":[Train_loss_gen_to_help_pred.detach().numpy()],
    "Train_loss_Discriminator_fool": [Train_loss_gen_to_fool_dis.detach().numpy()],
    "Train_total_loss_gen": [Train_total_loss_gen.detach().numpy()],
    "Dataset_fairness_Demographic_Parity": [Dataset_fairness_Demographic_Parity],
    "Report_DNN_Discriminator_Training_Accuracy": [Report_DNN_Discriminator_Training_Accuracy],
    "Report_DNN_Discriminator_Training_precision": [Report_DNN_Discriminator_Training_precision],
    "Report_DNN_Discriminator_Training_recall": [Report_DNN_Discriminator_Training_recall],
    "Report_DNN_Discriminator_Training_f1": [Report_DNN_Discriminator_Training_f1],
    "Report_DNN_Discriminator_Testing_Accuracy": [Report_DNN_Discriminator_Testing_Accuracy],
    "Report_DNN_Discriminator_Testing_precision": [Report_DNN_Discriminator_Testing_precision],
    "Report_DNN_Discriminator_Testing_recall": [Report_DNN_Discriminator_Testing_recall],
    "Report_DNN_Discriminator_Testing_f1": [Report_DNN_Discriminator_Testing_f1],
    "Report_Predictor_Training_Accuracy": [Report_Predictor_Training_Accuracy],
    "Report_Predictor_Training_precision": [Report_Predictor_Training_precision],
    "Report_Predictor_Training_recall": [Report_Predictor_Training_recall],
    "Report_Predictor_Training_f1": [Report_Predictor_Training_f1],
    "Report_Predictor_Testing_Accuracy": [Report_Predictor_Testing_Accuracy],
    "Report_Predictor_Testing_precision": [Report_Predictor_Testing_precision],
    "Report_Predictor_Testing_recall": [Report_Predictor_Testing_recall],
    "Report_Predictor_Testing_f1": [Report_Predictor_Testing_f1],
    "Report_Before_CatBoost_Prediction_Training_Accuracy": [Report_Before_CatBoost_Prediction_Training_Accuracy],
    "Report_Before_CatBoost_Prediction_Training_precision": [Report_Before_CatBoost_Prediction_Training_precision],
    "Report_Before_CatBoost_Prediction_Training_recall": [Report_Before_CatBoost_Prediction_Training_recall],
    "Report_Before_CatBoost_Prediction_Training_f1": [Report_Before_CatBoost_Prediction_Training_f1],
    "Report_Before_CatBoost_Prediction_Testing_Accuracy": [Report_Before_CatBoost_Prediction_Testing_Accuracy],
    "Report_Before_CatBoost_Prediction_Testing_precision": [Report_Before_CatBoost_Prediction_Testing_precision],
    "Report_Before_CatBoost_Prediction_Testing_recall": [Report_Before_CatBoost_Prediction_Testing_recall],
    "Report_Before_CatBoost_Prediction_Testing_f1": [Report_Before_CatBoost_Prediction_Testing_f1],
    "Report_After_CatBoost_Prediction_Training_Accuracy": [Report_After_CatBoost_Prediction_Training_Accuracy],
    "Report_After_CatBoost_Prediction_Training_precision": [Report_After_CatBoost_Prediction_Training_precision],
    "Report_After_CatBoost_Prediction_Training_recall": [Report_After_CatBoost_Prediction_Training_recall],
    "Report_After_CatBoost_Prediction_Training_f1": [Report_After_CatBoost_Prediction_Training_f1],
    "Report_After_CatBoost_Prediction_Testing_Accuracy": [Report_After_CatBoost_Prediction_Testing_Accuracy],
    "Report_After_CatBoost_Prediction_Testing_precision": [Report_After_CatBoost_Prediction_Testing_precision],
    "Report_After_CatBoost_Prediction_Testing_recall": [Report_After_CatBoost_Prediction_Testing_recall],
    "Report_After_CatBoost_Prediction_Testing_f1": [Report_After_CatBoost_Prediction_Testing_f1],
    "Report_Before_CatBoost_Sensitive_Training_Accuracy": [Report_Before_CatBoost_Sensitive_Training_Accuracy],
    "Report_Before_CatBoost_Sensitive_Training_precision": [Report_Before_CatBoost_Sensitive_Training_precision],
    "Report_Before_CatBoost_Sensitive_Training_recall": [Report_Before_CatBoost_Sensitive_Training_recall],
    "Report_Before_CatBoost_Sensitive_Training_f1": [Report_Before_CatBoost_Sensitive_Training_f1],
    "Report_Before_CatBoost_Sensitive_Testing_Accuracy": [Report_Before_CatBoost_Sensitive_Testing_Accuracy],
    "Report_Before_CatBoost_Sensitive_Testing_precision": [Report_Before_CatBoost_Sensitive_Testing_precision],
    "Report_Before_CatBoost_Sensitive_Testing_recall": [Report_Before_CatBoost_Sensitive_Testing_recall],
    "Report_Before_CatBoost_Sensitive_Testing_f1": [Report_Before_CatBoost_Sensitive_Testing_f1],
    "Report_After_CatBoost_Sensitive_Training_Accuracy": [Report_After_CatBoost_Sensitive_Training_Accuracy],
    "Report_After_CatBoost_Sensitive_Training_precision":[ Report_After_CatBoost_Sensitive_Training_precision],
    "Report_After_CatBoost_Sensitive_Training_recall": [Report_After_CatBoost_Sensitive_Training_recall],
    "Report_After_CatBoost_Sensitive_Training_f1": [Report_After_CatBoost_Sensitive_Training_f1],
    "Report_After_CatBoost_Sensitive_Testing_Accuracy": [Report_After_CatBoost_Sensitive_Testing_Accuracy],
    "Report_After_CatBoost_Sensitive_Testing_precision": [Report_After_CatBoost_Sensitive_Testing_precision],
    "Report_After_CatBoost_Sensitive_Testing_recall": [Report_After_CatBoost_Sensitive_Testing_recall],
    "Report_After_CatBoost_Sensitive_Testing_f1": [Report_After_CatBoost_Sensitive_Testing_f1],
    "Fairness_Equal_Odd_Before_CatBoost_prediction_Training": [Fairness_Equal_Odd_Before_CatBoost_prediction_Training],
    "Fairness_Equal_Odd_Before_CatBoost_prediction_Testing": [Fairness_Equal_Odd_Before_CatBoost_prediction_Testing],
    "Fairness_Equal_Odd_DNN_prediction_Training": [Fairness_Equal_Odd_DNN_prediction_Training],
    "Fairness_Equal_Odd_DNN_prediction_Testing": [Fairness_Equal_Odd_DNN_prediction_Testing],
    "Fairness_Equal_Odd_After_CatBoost_prediction_Training": [Fairness_Equal_Odd_After_CatBoost_prediction_Training],
    "Fairness_Equal_Odd_After_CatBoost_prediction_Testing": [Fairness_Equal_Odd_After_CatBoost_prediction_Testing],
    "Fairness_Equal_Opportunity_Before_CatBoost_prediction_Training": [Fairness_Equal_Opportunity_Before_CatBoost_prediction_Training],
    "Fairness_Equal_Opportunity_Before_CatBoost_prediction_Testing": [Fairness_Equal_Opportunity_Before_CatBoost_prediction_Testing],
    "Fairness_Equal_Opportunity_DNN_prediction_Training": [Fairness_Equal_Opportunity_DNN_prediction_Training],
    "Fairness_Equal_Opportunity_DNN_prediction_Testing": [Fairness_Equal_Opportunity_DNN_prediction_Testing],
    "Fairness_Equal_Opportunity_After_CatBoost_prediction_Training": [Fairness_Equal_Opportunity_After_CatBoost_prediction_Training],
    "Fairness_Equal_Opportunity_After_CatBoost_prediction_Testing": [Fairness_Equal_Opportunity_After_CatBoost_prediction_Testing],
    "Fairness_Demographic_Parity_Before_CatBoost_prediction_Training": [Fairness_Demographic_Parity_Before_CatBoost_prediction_Training],
    "Fairness_Demographic_Parity_Before_CatBoost_prediction_Testing": [Fairness_Demographic_Parity_Before_CatBoost_prediction_Testing],
    "Fairness_Demographic_Parity_DNN_prediction_Training": [Fairness_Demographic_Parity_DNN_prediction_Training],
    "Fairness_Demographic_Parity_DNN_prediction_Testing": [Fairness_Demographic_Parity_DNN_prediction_Testing],
    "Fairness_Demographic_Parity_After_CatBoost_prediction_Training": [Fairness_Demographic_Parity_After_CatBoost_prediction_Training],
    "Fairness_Demographic_Parity_After_CatBoost_prediction_Testing": [Fairness_Demographic_Parity_After_CatBoost_prediction_Testing],
    "Fairness_Treatment_Equality_Before_CatBoost_prediction_Training": [Fairness_Treatment_Equality_Before_CatBoost_prediction_Training],
    "Fairness_Treatment_Equality_Before_CatBoost_prediction_Testing": [Fairness_Treatment_Equality_Before_CatBoost_prediction_Testing],
    "Fairness_Treatment_Equality_DNN_prediction_Training": [Fairness_Treatment_Equality_DNN_prediction_Training],
    "Fairness_Treatment_Equality_DNN_prediction_Testing": [Fairness_Treatment_Equality_DNN_prediction_Testing],
    "Fairness_Treatment_Equality_After_CatBoost_prediction_Training": [Fairness_Treatment_Equality_After_CatBoost_prediction_Training],
    "Fairness_Treatment_Equality_After_CatBoost_prediction_Testing": [Fairness_Treatment_Equality_After_CatBoost_prediction_Testing],
        }

    df_temp_results = pd.DataFrame(results)
    df_results = pd.concat([df_results, df_temp_results], ignore_index=True)
    df_results.to_csv(Dataset_name+"_"+Date[:10]+"_abl_a0.csv")


    ####################################################################################################
    #                                  Update Alpha, Beta, Gamma 
    ####################################################################################################
    # Call the function after a training iteration
    learning_rate = 1.5
    # Assuming initial values for Alpha, Beta, Gamma and their respective Adam optimizer parameters
    alpha_m, alpha_v, beta_m, beta_v, gamma_m, gamma_v, t = 0, 0, 0, 0, 0, 0, 0
    
    # When calling the update_hyperparameters function
    Alpha, Beta, Gamma, alpha_m, alpha_v, beta_m, beta_v, gamma_m, gamma_v, t = update_hyperparameters(
        Alpha, Beta, Gamma, loss_gen_to_help_pred*100, cat_sen_bce_loss, dp_loss_predictor, learning_rate,
        alpha_m, alpha_v, beta_m, beta_v, gamma_m, gamma_v, t)

    Alpha, Beta, Gamma = float(Alpha.detach().numpy()),   float(Beta.detach().numpy()),    float(Gamma.detach().numpy()) 
    print(f" {count} Updated Alpha: {Alpha}, Beta: {Beta}, Gamma: {Gamma}")

    class_pred = np.where(predictor( generator(X_Test_tensor)) >= 0.5, 1, 0)
    sen_pred = np.where(predictor( generator(X_Test_tensor)) >= 0.5, 1, 0)
    pred_accuracy = accuracy_score(class_pred, y_Test_tensor.numpy())
    dp_loss_predictor = demographic_parity_loss(sen_pred, sen_Test_tensor.numpy())

    # Alpha, Beta, Gamma = float(Alpha.detach().numpy(), Beta.detach().numpy(), Gamma.detach().numpy()




    print(time)

/tmp/ipykernel_1641/3273344808.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampled_df = pd.concat([sampled_df, sampled_group])


Dataset_count 1 count: 1 iter: <built-in function iter> Sensitive: 1 Class: 0  a:0.33 b:0.33 r:0.3399999999999999
|..